In [1]:
## preprocessing
##remove special characters except . , ? ! (check for , in test data. if it isn't there remove in train)
#remove hyperlinks (replace with "hyperlink"), emojis (replace with "emoji"), ellipses, . . ., 
# lower case all
import re
import random
import json

In [28]:
#Friends Script Data

In [3]:
f = open('twitter_reddit_chat/friends-final.txt')

In [4]:
friendsdata = f.read().split('\n')

In [5]:
dias = []
for i,x in enumerate(friendsdata[:-1]):
    x = x.split('\t')[5].strip()
    if x is not '':
        dias.append(x).lower()
dias = dias[1:] # remove header
dias = list(set(dias))

In [ ]:
def processFile(file)
    f = open(file)'twitter_reddit_chat/MovieCorpus.txt')
    tempdat = f.read().split('\n')
    newdat = []
    for i,x in enumerate(tempdat):
        if x is not '':
            newdat.append(x).lower()
    return newdat

In [7]:
# MovieCorpus.txt

In [8]:
moviecorp = processFile('twitter_reddit_chat/MovieCorpus.txt')

In [12]:
# TwitterLowerAsciiCorpus.txt

In [15]:
tweetcorp = processFile('twitter_reddit_chat/TwitterLowerAsciiCorpus.txt')

In [16]:
## ijcnlp

In [ ]:
dailydia = processFile('twitter_reddit_chat/ijcnlp_dailydialog/dialogues_text.txt')

In [20]:
## conv ai2

In [ ]:
convai2data = processFile('twitter_reddit_chat/convai2/train_none_original.txt')

In [26]:
## quac data

In [27]:
quacjson = json.load(open("QA/quac_train_v0.2.json"))


In [28]:
quacjson['data'][0].keys()

dict_keys(['paragraphs', 'section_title', 'background', 'title'])

In [29]:
quacdata = []
for x in quacjson['data']:
    for t in x['paragraphs']:
        for z in t['qas']:
            q = z['question'].strip().lower()
            a = z['orig_answer']['text'].strip().lower()
            if q is not '':
                quacdata.append(q)
            if a is not '' and a is not 'cannotanswer':
                quacdata.append(a)


In [31]:
## combine all data, perform preprocessing

In [32]:
data = quacdata + convai2data + dailydia + tweetcorp + dias + moviecorp
data = list(set(data))

In [23]:
## remove noise and special chars
http_sub=re.compile(r"http\S+")

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001f199-\U0001f918"
        u"\U0001f0cf-\U000fec2c"
        #u"\u0101-\uffffe3"
        u"\u00002026" #ellipsis
        "0123456789@#$$%^&*(){}|+_\-=\"\';\?\.!,:/<>~`\]\["                   "]+", flags=re.UNICODE) # 2 datasets -  with and without removing ?,.,,,!

multispace = re.compile(" +")

In [16]:
## remove multiple instances of punctuation - incase including punctuation, for a different type of model
multiq = re.compile("\?+")
multiex = re.compile("!+")
multidot = re.compile("\.+")
multicomma = re.compile(",+")

def remrep(t_t,punct):
    t_t = t_t.split(punct)
    t_t = [x.strip() for i, x in enumerate(t_t) if x.strip() != '']
    punct = ' '+punct+' '
    t_t = punct.join(t_t)
    return t_t

def subpuncts(t_t):
    t_t = multiq.sub('?',t_t)
    t_t = multiex.sub('!',t_t)
    t_t = multidot.sub('.',t_t)
    t_t = multicomma.sub('.',t_t)

    t_t = remrep(t_t,'?')
    t_t = remrep(t_t,'!')
    t_t = remrep(t_t,'.')
    t_t = remrep(t_t,',')
    
    return t_t

def denoise(dat):
    for i in range(len(dat)):
        temp = dat[i]
        temp = emoji_pattern.sub('',temp)
        temp = http_sub.sub('<link>',temp)
        temp = multispace.sub(' ',temp)

        temp = temp.strip()
        temp = temp + ' <eos>'    
        ## punctuation processing
        temp = subpuncts(temp)
        temp = temp.rsplit(' ', 1)[0]
        dat[i] = temp
        return dat

In [ ]:
## denoise data
data = denoise(data)

In [12]:
## if needed split data into questions, statements - process them separateley
q_end = []
q_middle = []
q_all = []
stats = []

In [13]:
for x in data:
    if '?' in x:
        qpos = x.index('?')
        #x = x.replace('?','').strip()
        q_all.append(x)
        if qpos == (len(x) - 1):
            q_end.append(x)
            continue
        q_middle.append(x)
        continue
    stats.append(x)

In [35]:
#random.shuffle(q_all)
#random.shuffle(stats)

In [51]:
## denoise statements
stats = denoise(stats)
## denoise questions
q_all = denoise(q_all)

In [53]:
## equalize lengths of split
q_all = q_all[:200000]
stats = stats[:200000]

In [56]:
alldat = q_all + stats

In [57]:
## shuffle data
random.shuffle(alldat)

In [60]:
with open('alldat_no_punct.txt','w') as f:
    for x in alldat:
        #print(x)
        if x is None or x is '':
            continue
        f.write(x)
        f.write('\n')
f.close()

In [ ]:
## test dat Processing

In [24]:
f = open('test-inputs.txt')
testdata = f.read().split('\n')

In [25]:
## denoise testdata
testdat = denoise(testdata)

In [27]:
with open('testdat_punct.txt','w') as f:
    for x in testdat:
        ## continue if null string
        if x is None or x is '':
            continue
        f.write(x)
        f.write('\n')
f.close()